# 2025.06.04

In [ ]:
from dotenv import load_dotenv
load_dotenv() # If True, reading successful.
# Make values environment variables temporarily.

In [ ]:
import os


key = os.getenv("HUGGINGFACE_API_KEY")
# key = os.getenv("HUGGINGFACE_API_KEYsssda") # None
print(key)

In [ ]:
key = os.environ['HUGGINGFACE_API_KEY']
# key = os.environ['HUGGINGFACE_API_KEYs'] # KeyError
print(key)

## Fine Tuning

### Naver Movie comments Classification

#### Huggingface Dataset Package

* `pip install datasets`
* https://huggingface.co/datasets
* https://github.com/huggingface/datasets

#### Loading

* `load_dataset('dastaset_name')`

In [ ]:
%pip install --upgrade datasets

In [ ]:
from datasets import load_dataset

nsmc = load_dataset('e9t/nsmc', trust_remote_code = True)   # 
type(nsmc)

In [ ]:
# Dataset: actual dataset
# DatasetDict: dictionary of dataset collection(train/valid/test)

nsmc

In [ ]:
trainset = nsmc['train']
testset = nsmc.get('test')
trainset, testset

In [ ]:
trainset['document'][:5], trainset['label'][:5], trainset['id'][:5]

In [ ]:
df = trainset.to_pandas()
df.head()

In [ ]:
# datasets.Dataset.from_extension(dataset_name)
import datasets

d = datasets.Dataset.from_pandas(df.head(100))
d

#### sampling

* train: 10000

* test: 5000

In [ ]:
sample_train = trainset.shuffle().select(range(10000))
sample_test = testset.shuffle().select(range(5000))
# sample_test = testset.shuffle()[:5000]

In [ ]:
print(sample_train, sample_test, sep = '\n')

#### Model, Tokenizer by AutoClass

* https://huggingface.co/docs/transformers/model_doc/auto

In [ ]:
%pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = 'beomi/kcbert-base'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2)

In [ ]:
# beomi/kcbert-base -> Feature Extraction Model(not for Classification)

# Message: You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# L Untrained Estimator

In [ ]:
print(model)

#### pytorch Dataset, DataLoader

In [ ]:
train_X, test_X, train_y, test_y = sample_train['document'], sample_test['document'], sample_train['label'], sample_test['label']

In [ ]:
# Tokenization
train_encoding = tokenizer(train_X, return_tensors = 'pt', padding = True) # padding standard?
test_encoding = tokenizer(test_X, return_tensors = 'pt', padding = True)

In [ ]:
train_encoding.keys()
train_encoding['input_ids'].shape

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):

    def __init__(self, comments, labels):
        """
        Args:
            comments(dict)
            labels(list)
        """
        self.comments = comments
        self.labels = labels


    def __len__(self):
        return(len(self.labels))
    
    def __getitem__(self, idx):
        """
        to look up a single sequence in the batch
        BERT Input Format -> (input_ids, token_type_ids, attention_mask)
        Args(int):
            idx(int)
        Returns:
            dict - input_ids, token_type_ids, attention_mask, label
        """
        data = {key:val[idx] for key, val in self.comments.items()}
        data['labels'] = torch.tensor([self.labels[idx]], dtype = torch.int64)
        return data

In [ ]:
train_encoding

In [ ]:
train_set = NSMCDataset(train_encoding, train_y) 
test_set = NSMCDataset(test_encoding, test_y)

In [ ]:
len(train_set), len(test_set)

In [ ]:
train_set[0]

#### Train

* `TrainingArguments`, `Trainer` -> Convenience

In [ ]:
from transformers import TrainingArguments, Trainer

N_EPOCHS = 1
BATCH_SIZE = 64

train_args = TrainingArguments(
    output_dir = 'models/nsmc',
    num_train_epochs = N_EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,

    eval_strategy = 'epoch',  # 'no', 'step', 'epoch
    save_strategy = 'epoch',

    save_total_limit = 1,
    load_best_model_at_end = True,   # Save and Load. subject to: eval_strategy = save_strategy

    metric_for_best_model = 'eval_loss',
    greater_is_better = False,      # like NMSE

    report_to = 'none'
)

In [ ]:
import evaluate

# Performance Evaluation
acc_fn = evaluate.load('accuracy')   # f1, recall, precision, etc.
acc_fn

In [ ]:
pred = torch.tensor([0, 1, 0, 1])
ref = torch.tensor([0, 1, 0, 0])

acc_fn.compute(predictions = pred, references = ref)

In [ ]:
def compute_metrics(pred):
    """
    to get predicted values and evaluate performance while training
    Args:
        pred(EvalPrediction): Predicted values with ground truths
    Returns:
        dict - dict(score_name = score) 
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(dim = -1)
    metrics1 = evaluate.load("accuracy")
    metrics2 = evaluate.load("f1")

    acc = metrics1.compute(predictions = preds, references = labels)
    f1 = metrics2.compute(predictions = preds, references = labels)
    return {"Accuracy": acc, "F1 Score": f1}

In [ ]:
trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset = train_set,   # torch.utils.data.Dataset   # trainer.train()
    eval_dataset = testset,     # trainer.evaluate()
    compute_metrics = compute_metrics   # default = loss function
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# save: tokenizer and model always same path
save_path = 'models/nsmc'
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

In [ ]:
# load
from transformers import AutoTokenizer, AutoModelForSequenceClassification

load_tokenizer = AutoTokenizer.from_pretrained(save_path)   # local path
load_model = AutoModelForSequenceClassification.from_pretrained(save_path)

#### Prediction

In [ ]:
sentence = ["이걸 영화라고 만든 거냐?", "아무 기대 없이 봤는데 재미있네.", "내가 감독이어도 이것보다 재미있게 만들겠다.", "시간이 어떻게 가는 줄 모르고 봤다."]

In [ ]:
from transformers import pipeline

pipe = pipeline(task = 'text-classification', tokenizer = load_tokenizer, model = load_model)
result = pipe(sentence)
result

In [ ]:
from huggingface_hub import login

login(key)

In [ ]:
model_id = 'kcbert-nsmc-10000'
load_tokenizer.push_to_hub(model_id)
load_model.push_to_hub(model_id)